# Rotation Representation

Rotation is quite important in human motion (and pose) representation. Here we aim to provide a brief introduction to rotation representation and some related concepts. We won't go into the very details of the mathematics, but we will provide some intuitions and examples.

Here is the table of the content:

- Eular Angles
- Axis-Angle
- Rotation Matrix and 6D Vector
- Quaternion

## Tutorials

### Environment Preparation

In [1]:
# Packages you may use very often.
import torch
import numpy as np
from smplx import SMPL
from pytorch3d import transforms  # You may use this package when performing rotation representation transformation.

# Things you don't need to care about. They are just for driving the tutorials.
from lib.utils.path_manager import PathManager
from lib.viewer.wis3d_utils import HWis3D as Wis3D
from lib.skeleton import Skeleton_SMPL24

pm = PathManager()

In this chapter, we will visualize the effect of the rotation through the facing director of a digital human. You can just ignore things related to `SMPL` model, and we will introduce it in [SMPL-basic](SMPL_basic.ipynb).

In [ ]:
# We will use a digital human to represent the direction change.
body_model = SMPL(
        model_path = pm.inputs / 'body_models' / 'smpl',
        gender     = 'neutral',
    )

### Simple Transformation

You may only have data in one specific rotation representation, and you want to convert it to another representation. You can easily achieve this by using `pytorch3d` package. Here we give an example of converting common other representation to axis angle.

In [ ]:
def to_axis_angle(
    rotation   : torch.Tensor,
    rot_format : str,
):
    """ Utils to transform the rotation representation. """

    # 1. Transform the rotation representation to axis angle.
    if rot_format == "euler_angle":
        res = transforms.euler_angles_to_matrix(rotation, "XYZ").reshape(-1, 1, 3)
    elif rot_format == "axis_angle":
        res = rotation.reshape(-1, 1, 3)
    elif rot_format == "rotation_matrix":
        res = transforms.matrix_to_axis_angle(rotation).reshape(-1, 1, 3)
    elif rot_format == "rotation_6d":
        rotation = transforms.rotation_6d_to_matrix(rotation)
        res = transforms.matrix_to_axis_angle(rotation).reshape(-1, 1, 3)
    elif rot_format == "quaternion":
        res = transforms.quaternion_to_matrix(rotation).reshape(-1, 1, 3)
    else:
        raise NotImplementedError(f"Unknown rotation format: {rot_format}")

    return res

### Euler Angle

You can check this video for an intuitive understanding of Euler angles:
 [Euler angles - Insights | YouTube](https://www.youtube.com/watch?v=qqlLshaHqeE)

### Axis Angle

An axis angle is represented as a 3D vector, let's note it as $[x, y, z]$. The direction of the vector represents the rotation axis, and the length of the vector represents the rotation angle.

For example, a vector $\vec{r} = [x, y, z]$ represents a rotation around the axis $\frac{\vec{r}}{||\vec{r}||}$ in radians $||\vec{r}||$.

### Rotation Matrix and 6D Vector

Rotation matrix is a 3x3 matrix that can be used to represent a rotation in 3D space.

If we only leave the first two columns of the rotation matrix, we can get a 3x2 matrix. We can flatten this matrix to a 6D vector.

So, why do we need to use 6D vector to represent a rotation matrix? Check this: [On the Continuity of Rotation Representations in Neural Networks](http://arxiv.org/abs/1812.07035).

### Quaternion

Quaternion is a 4D vector that can be used to represent a rotation in 3D space.

## References

- [图形学笔记.旋转相关 | 二圈妹](https://zhuanlan.zhihu.com/p/66157419)